In [1]:
pip install streamlit fastapi uvicorn openai whisper langchain faiss-cpu pyttsx3 yfinance

Note: you may need to restart the kernel to use updated packages.


In [50]:
# import yfinance as yf

# def get_tech_stock_data(tickers: list[str]):
#     data = {}
#     for ticker in tickers:
#         stock = yf.Ticker(ticker)
#         hist = stock.history(period="2d")
#         if len(hist) >= 2:
#             yesterday = hist.iloc[-2]['Close']
#             today = hist.iloc[-1]['Close']
#             change_pct = ((today - yesterday) / yesterday) * 100
#             data[ticker] = {
#                 'today': today,
#                 'yesterday': yesterday,
#                 'change_pct': round(change_pct, 2)
#             }
#     return data

In [51]:
# from api_agent import get_tech_stock_data


# if __name__ == "__main__":
#     tickers = ['TCS.NS', 'INFY.NS', 'RELIANCE.NS']
#     print(get_tech_stock_data(tickers))

In [52]:
# from fastapi import FastAPI, Query
# from typing import List
# # from api_agent import get_tech_stock_data

# app = FastAPI(title="API Agent Service", description="Fetch Indian tech stock data")

# @app.get("/tech-stocks")
# def fetch_tech_stocks(tickers: List[str] = Query(..., description="List of NSE tickers")):
#     """
#     Example: /tech-stocks?tickers=TCS.NS&tickers=INFY.NS
#     """
#     result = get_tech_stock_data(tickers)
#     return {"status": "success", "data": result}

In [53]:
# !cd agents/api_agent && uvicorn api_service:app --reload --port 8001

In [54]:
# print("Processing ticker:", ticker)
# print("History data:", hist)

In [55]:
# import yfinance as yf

# ticker = yf.Ticker("TCS.NS")
# hist = ticker.history(period="2d")
# print(hist)


In [56]:
# @app.get("/tech-stocks")
# def fetch_tech_stocks(tickers: List[str] = Query(default=["TCS.NS", "INFY.NS"])):
#     result = get_tech_stock_data(tickers)
#     return {"status": "success", "data": result}


In [57]:
# import requests

# url = "http://localhost:8001/tech-stocks"
# response = requests.get(url)
# print(response.json())


In [58]:
import os
os.environ["ALPHAVANTAGE_API_KEY"] = "your_actual_key"


In [8]:
%%writefile api_agent.py
import yfinance as yf
import time
import random
import logging

logging.basicConfig(level=logging.INFO)

def get_tech_stock_data(tickers):
    stock_data = {}
    for ticker in tickers:
        logging.info(f"Fetching {ticker}")
        for i in range(10):  # 5 retries
            try:
                stock = yf.Ticker(ticker)
                hist = stock.history(period="1d", interval= "1m")
                if hist.empty:
                    raise Exception("No data returned")
                today = hist["Close"].iloc[-1]
                # yesterday = hist["Close"].iloc[-2]
                # change = today - yesterday
                # pct_change = (change / yesterday) * 100
                stock_data[ticker] = {
                    "today": round(today,2),
                    # "yesterday": round(yesterday,2),
                    # "change": round(change,2),
                    # "percent_change": round(pct_change,2)
                }
                break
            except Exception as e:
                logging.warning(f"Error on {ticker}, retry {i+1}: {e}")
                time.sleep(random.uniform(2,5))
        else:
            stock_data[ticker] = {"error": "Failed after retries"}
    return stock_data

# if __name__ == "__main__":
#     tickers = ['TCS.NS', 'INFY.NS', 'RELIANCE.NS']
#     print(get_tech_stock_data(tickers))


Overwriting api_agent.py


In [6]:
%%writefile api_service.py
from fastapi import FastAPI, Query
from typing import List
from api_agent import get_tech_stock_data

app = FastAPI()

@app.get("/tech-stocks")
def tech_stocks(tickers: List[str] = Query(...)):
    return get_tech_stock_data(tickers)


Overwriting api_service.py


In [9]:
from api_agent import get_tech_stock_data

if __name__ == "__main__":
    tickers = ['RELIANCE.NS']
    print(get_tech_stock_data(tickers))

INFO:root:Fetching RELIANCE.NS


{'RELIANCE.NS': {'error': 'Failed after retries'}}


In [80]:
pip install fastapi uvicorn nest_asyncio pyngrok yfinance

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# !ngrok config add-authtoken 2xe5ACQwWWtWx8tN85NTgPcrnOX_yM5fMKK7uL6BCgFL22Hg

Authtoken saved to configuration file: C:\Users\ravik\AppData\Local/ngrok/ngrok.yml


In [64]:
from pyngrok import ngrok
import nest_asyncio
import uvicorn

nest_asyncio.apply()

public_url = ngrok.connect(8000)
print("🔗 Public URL:", public_url)

uvicorn.run("api_service:app", host="0.0.0.0", port=8000)


INFO:pyngrok.ngrok:Opening tunnel named: http-8000-59f52960-c110-4cb8-bd60-d93952ce7920


🔗 Public URL: NgrokTunnel: "https://a92f-2409-40c4-197-5a85-ac0c-8217-3619-256e.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [17092]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [17092]
